## Задача кластеризации
### Выбор оптимального расположения рекламных баннеров на основе информации о перемещениях потенциальных клиентов туристической компании

импорт библиотек

In [537]:
import pandas as pd

In [538]:
import numpy as np

In [539]:
# считываем файл с данными
# удаляем пробелы внутри данных (после разделителя)
# сразу удаляем строки, где есть пропуски данных
data = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True, low_memory=False).dropna()

In [540]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [541]:
data.shape

(396634, 6)

In [542]:
data.keys()

Index([u'id    ', u'user_id ', u'venue_id ', u'latitude      ',
       u'longitude     ', u'created_at      '],
      dtype='object')

В названиях столбцов есть пробелы. Их нужно удалить:

In [543]:
data.columns = [col.strip() for col in data.columns]

#2 других варианта
#data.columns = [data.keys()[i].strip() for i in range(len(data.keys()))]
#data.columns = [data.keys()[i].replace(' ', '') for i in range(len(data.keys()))]

In [544]:
data.keys()

Index([u'id', u'user_id', u'venue_id', u'latitude', u'longitude',
       u'created_at'],
      dtype='object')

Импортируем классификатор-кластеризатор MeanShift. Он основан на отслеживании градиента плотности распределения. Кластеризация начинается из каждой точки и движется в сторону увеличения градиента до локального максимума. Происходит как-бы конденсация с центрами в локальных максимумах.

In [545]:
from sklearn.cluster import MeanShift

In [546]:
# Формируем вектор для обучения: вырезаем координаты и ограничиваемся 100000 строк для сокращения времени вычислений
X = data[['latitude','longitude']][:100000]

In [547]:
# формируем кластеризатор, обучаем, считываем метки кластеров, координаты центров кластеров и подсчитываем число кластеров 

ms = MeanShift(bandwidth=0.1)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

number of estimated clusters : 3230


In [548]:
# файл координат центров (видим, что получилось 3230 кластеров)
cluster_centers.shape

(3230L, 2L)

In [549]:
# тестируем метки классов
print labels[:100]

[   5    7   30   66    1   23    0    1    2    8    1    2    1  137  237
   11    2    1  504   45   22    1    5   32   47   11    4 1004   51   48
    3    3  397  915    0    0   38 2213    1    6    1   79    1    6  145
   10  612  267   10   14   10   10    4   30  353   32   12    3  190   28
  353   20  416  188    3   59 1013    1   12    1    1  211    0  133   94
    1    1    1    1    1  151    6  481  374  147    8    3  359    1   23
    1    1  261    6  674    5  512    3   10    0]


In [550]:
# тест
labels_unique

array([   0,    1,    2, ..., 3227, 3228, 3229], dtype=int64)

In [551]:
# тест
len(labels)

100000

In [552]:
# выведем координаты первых 100 кластеров для визуализации на https://www.mapcustomizer.com через функцию Bulk Entry
for i in range(100):
    print cluster_centers[i][0], cluster_centers[i][1]

40.7177163973 -73.9918354199
33.4494380502 -112.00213969
33.4463802704 -111.901887562
41.8782437797 -87.6298433623
37.6886815741 -122.409330374
38.8861652156 -77.0487833307
33.3573445623 -111.822654108
33.7666362322 -84.3932891848
42.3632186398 -71.0736876086
47.6062447174 -122.332043826
36.117229143 -115.171073423
34.0603975546 -118.248709027
44.9779478203 -93.2673008853
30.267183617 -97.7431192813
40.76687624 -73.8333534905
39.7358301526 -104.986580428
39.951680373 -75.1627359239
34.0354869531 -118.438997719
32.9808933822 -117.078117978
32.8030205353 -96.7698974349
37.3478711439 -121.947287224
28.5435015466 -81.3764286229
32.7113444339 -117.153638748
32.2217131518 -110.926535153
34.1274022191 -118.351883725
29.7626977547 -95.3823137047
43.0405328168 -87.914332566
33.8173064339 -117.891249171
37.390292432 -122.087286436
25.78581242 -80.2179380368
45.5234832146 -122.676280421
33.5697346035 -112.314015838
33.6743026598 -117.858789268
-6.21276665639 106.844659812
27.9494460796 -82.465073

In [657]:
%%time
# создадим ДатаФрейм, где в первом столбце посчитаем число точек в каждом кластере от 0 до его окончания
# а два других столбца - координаты этих центров (широта и долгота)

clusters = pd.DataFrame({'label_counts':[sum(labels==label) for label in np.unique(labels)],
                         'latitude':cluster_centers[:, 0],
                         'longitude':cluster_centers[:, 1]})

Wall time: 10min 2s


In [683]:
%%time
# альтернативный вариант предыдущего, который работает 5 ms(!) вместо 10 мин
# через метод для ДатаФрейм .value_counts()

clusters = pd.DataFrame({'label_counts':pd.DataFrame({'counts':labels})['counts'].value_counts().sort_index(),
                         'latitude':cluster_centers[:, 0],
                         'longitude':cluster_centers[:, 1]})

Wall time: 4 ms


In [554]:
# тест на число центров
len(cluster_centers)

3230

In [677]:
clusters

,label_counts,latitude,longitude
0,12506,40.717716,-73.991835
1,4692,33.449438,-112.002140
2,3994,33.446380,-111.901888
3,3363,41.878244,-87.629843
4,3526,37.688682,-122.409330
5,2409,38.886165,-77.048783
6,2297,33.357345,-111.822654
7,1601,33.766636,-84.393289
8,1526,42.363219,-71.073688
9,1378,47.606245,-122.332044


In [556]:
# альтернативный подсчет числа точек в кластере

labels_pd = pd.DataFrame({'label':labels})

labels_pd.label.value_counts()

0       12506
1        4692
2        3994
4        3526
3        3363
5        2409
6        2297
7        1601
8        1526
9        1378
10       1298
11       1081
13       1007
12       1006
31        907
16        870
15        868
18        808
19        807
22        754
23        747
21        722
14        714
28        679
25        656
17        645
20        612
30        594
26        580
27        577
        ...  
2790        1
2726        1
2662        1
2598        1
2470        1
2638        1
2406        1
2278        1
2257        1
2321        1
2385        1
2449        1
2982        1
3046        1
3110        1
3174        1
3214        1
3150        1
3086        1
3022        1
2958        1
2894        1
2217        1
2830        1
2281        1
2766        1
2345        1
2702        1
2409        1
3166        1
Name: label, dtype: int64

In [557]:
# оставляем кластеры для реального рассмотрения в дальнейшем - это кластеры с числом точек > 15 (т.е. убираем мелкие) 
# и для наглядности сортируем по числу точек в кластере

clusters_real = clusters[clusters.label_counts>15].sort_values('label_counts', ascending=False)

In [558]:
clusters_real

,label_counts,latitude,longitude
0,12506,40.717716,-73.991835
1,4692,33.449438,-112.002140
2,3994,33.446380,-111.901888
4,3526,37.688682,-122.409330
3,3363,41.878244,-87.629843
5,2409,38.886165,-77.048783
6,2297,33.357345,-111.822654
7,1601,33.766636,-84.393289
8,1526,42.363219,-71.073688
9,1378,47.606245,-122.332044


In [559]:
# формируем датафрейм со списком и координатами офисов 

list_offices = pd.DataFrame({'City':['Los Angeles', 'Miami', 'London', 'Amsterdam', 'Beijing', 'Sydney'],
                            'latitude':[33.751277, 25.867736, 51.503016, 52.378894, 39.366487, -33.868457],
                            'longitude':[-118.188740, -80.324116, -0.075479, 4.885084, 117.036146, 151.205134]})

In [560]:
list_offices

,City,latitude,longitude
0,Los Angeles,33.751277,-118.188740
1,Miami,25.867736,-80.324116
2,London,51.503016,-0.075479
3,Amsterdam,52.378894,4.885084
4,Beijing,39.366487,117.036146
5,Sydney,-33.868457,151.205134


In [561]:
# делаем датафрейм с расстояниями от всех кластеров до всех офисов
# для простоты землю вблизи иссследуемой территории считаем плоской (евклидова метрика) и считаем в градусах 
# (для перехода в км нужно учесть, что в 1 град ~ 100 км)

distances = pd.DataFrame({City: \
                          np.sqrt((clusters_real.latitude - list_offices.latitude[list_offices['City']==City].values)**2 + \
                                  (clusters_real.longitude - list_offices.longitude[list_offices['City']==City].values)**2) \
                          for City in list_offices['City']})

In [562]:
# альтернативный вариант предыдущей операции

distances = pd.DataFrame({list_offices.City[i]:np.sqrt((clusters_real.latitude - list_offices.latitude[i])**2 + \
(clusters_real.longitude - list_offices.longitude[i])**2) for i in range(len(list_offices))})

In [565]:
distances

,Amsterdam,Beijing,London,Los Angeles,Miami,Sydney
0,79.734255,191.032760,74.699066,44.742571,16.143720,237.227259
1,118.410081,229.114704,113.373317,6.193959,32.572679,271.679532
2,118.311609,229.014565,113.274833,6.294241,32.474475,271.581649
4,128.139261,239.451355,123.111360,5.772048,43.713842,282.816724
3,93.108944,204.681402,88.081797,31.621097,17.598580,250.558794
5,83.037415,194.085524,78.000478,41.459174,13.424131,239.568541
6,118.247687,228.937678,113.210850,6.378262,32.376722,271.482828
7,91.197829,201.507260,86.163056,33.795454,8.885426,245.114510
8,76.616243,188.133702,71.584087,47.895654,18.912202,234.987538
9,127.306621,239.509966,122.318651,14.461227,47.299353,285.413235


In [563]:
# тест для типов величин (1)
{City:list_offices.longitude[list_offices['City']==City] for City in list_offices['City']}

{'Amsterdam': 3    4.885084
 Name: longitude, dtype: float64, 'Beijing': 4    117.036146
 Name: longitude, dtype: float64, 'London': 2   -0.075479
 Name: longitude, dtype: float64, 'Los Angeles': 0   -118.18874
 Name: longitude, dtype: float64, 'Miami': 1   -80.324116
 Name: longitude, dtype: float64, 'Sydney': 5    151.205134
 Name: longitude, dtype: float64}

In [564]:
# тест для типов величин (2)
{list_offices.City[i]:list_offices.longitude[i] for i in range(len(list_offices))}

{'Amsterdam': 4.885084,
 'Beijing': 117.036146,
 'London': -0.075479000000000004,
 'Los Angeles': -118.18874,
 'Miami': -80.324116000000004,
 'Sydney': 151.20513399999999}

In [566]:
# определяем расстояния от всех кластеров до ближайшего офиса (то есть минимальное)
# сортируем по расстоянию (по возрастанию) и "отрезаем" 20 первых (минимальных)
# обращаю внимание, что индексы объектов Sieries сохраняются по всем операциям и равны первоначальным в clusters

dist_20_min = distances.min(axis=1).sort_values()[:20]
dist_20_min

413    0.007835
373    0.009353
405    0.022674
58     0.050058
51     0.070848
29     0.134109
166    0.167406
92     0.188876
87     0.195779
42     0.211811
285    0.222233
315    0.271301
119    0.294979
55     0.302270
27     0.304731
11     0.314884
32     0.338810
158    0.340846
17     0.378688
47     0.386706
dtype: float64

In [686]:
# индексы первых 20 отобраных центров для размещения рекламных баннеров

dist_20_min.index

Int64Index([413, 373, 405,  58,  51,  29, 166,  92,  87,  42, 285, 315, 119,
             55,  27,  11,  32, 158,  17,  47],
           dtype='int64', name=u'cluster number ')

In [687]:
# определяем ближайшие офисы по этим индексам
# для чего сначала определяем офисы с минимальными расстояниями (через аналог argmin)
# а затем по индексам вырезаем нужные офисы

dist_20_cities_argmin = distances.idxmin(axis=1)
dist_20_cities_argmin[dist_20_min.index]

cluster number 
413         Sydney
373      Amsterdam
405          Miami
58          London
51     Los Angeles
29           Miami
166          Miami
92           Miami
87     Los Angeles
42     Los Angeles
285    Los Angeles
315          Miami
119    Los Angeles
55           Miami
27     Los Angeles
11     Los Angeles
32     Los Angeles
158          Miami
17     Los Angeles
47     Los Angeles
dtype: object

In [569]:
# собираем предыдущие 2 операции в наглядный итоговый датафрейм с минимальными дистанциями до офисов и самими офисами, 
# плюс индексы-номера кластеров

clusters_min_dist_cities = pd.DataFrame({'dist':dist_20_min, 
                                        'city':dist_20_cities_argmin[dist_20_min.index]})
clusters_min_dist_cities.index.name = 'cluster number '
clusters_min_dist_cities

,city,dist
cluster number,,
413,Sydney,0.007835
373,Amsterdam,0.009353
405,Miami,0.022674
58,London,0.050058
51,Los Angeles,0.070848
29,Miami,0.134109
166,Miami,0.167406
92,Miami,0.188876
87,Los Angeles,0.195779


In [570]:
# номер кластера с минимальным расстоянием до офиса

ind_min = dist_20_min.index[0]
ind_min

413

In [571]:
# тест координаты
clusters_real.ix[ind_min]['latitude']

-33.860630428571433

In [572]:
# запишем координаты этого офиса

with open('task1_ans.txt', 'w') as ans:
    ans.write(str(clusters_real.ix[ind_min]['latitude']) + ' ' +
              str(clusters_real.ix[ind_min]['longitude']))

# Работа завершена

## далее некоторые тесты-исследования

In [573]:
clusters_real.ix[ind_min]['latitude']

-33.860630428571433

In [574]:
list_offices[list_offices['City']=='London']

,City,latitude,longitude
2,London,51.503016,-0.075479


In [575]:
list_offices[list_offices['City']=='London'].latitude

2    51.503016
Name: latitude, dtype: float64

In [576]:
list_offices.latitude[list_offices['City']=='London']

2    51.503016
Name: latitude, dtype: float64

In [577]:
a = list_offices[list_offices['City']=='London'].latitude

In [578]:
type(a)

pandas.core.series.Series

In [579]:
a.iloc[0]

51.503016000000002

In [580]:
clusters_min_dist_cities.ix[11]
# ix = выбор по первоночально сформировнным меткам
# iloc - выбор по текущему номеру строки (и, видимо, колонки)

city    Los Angeles
dist       0.314884
Name: 11, dtype: object

In [581]:
clusters_min_dist_cities.iloc[11]

city       Miami
dist    0.271301
Name: 315, dtype: object

In [583]:
np.unique(labels)

array([   0,    1,    2, ..., 3227, 3228, 3229], dtype=int64)

In [585]:
%time
[sum(labels==label) for label in np.unique(labels)]

Wall time: 0 ns


[12506,
 4692,
 3994,
 3363,
 3526,
 2409,
 2297,
 1601,
 1526,
 1378,
 1298,
 1081,
 1006,
 1007,
 714,
 868,
 870,
 645,
 808,
 807,
 612,
 722,
 754,
 747,
 539,
 656,
 580,
 577,
 679,
 564,
 594,
 907,
 449,
 502,
 452,
 104,
 431,
 410,
 388,
 400,
 369,
 367,
 384,
 347,
 345,
 342,
 314,
 273,
 314,
 316,
 355,
 281,
 336,
 293,
 271,
 246,
 263,
 258,
 254,
 243,
 229,
 291,
 182,
 155,
 137,
 193,
 186,
 197,
 196,
 189,
 191,
 187,
 203,
 178,
 192,
 169,
 173,
 153,
 157,
 199,
 220,
 164,
 162,
 155,
 126,
 152,
 56,
 100,
 141,
 142,
 190,
 117,
 138,
 133,
 135,
 131,
 134,
 122,
 130,
 109,
 132,
 135,
 135,
 120,
 119,
 116,
 110,
 91,
 114,
 114,
 79,
 98,
 112,
 74,
 110,
 67,
 126,
 106,
 110,
 74,
 103,
 102,
 101,
 100,
 85,
 99,
 72,
 99,
 118,
 95,
 98,
 84,
 27,
 99,
 95,
 97,
 97,
 92,
 91,
 103,
 71,
 95,
 87,
 99,
 95,
 33,
 85,
 65,
 121,
 78,
 83,
 81,
 79,
 101,
 79,
 61,
 79,
 78,
 42,
 84,
 78,
 68,
 57,
 63,
 76,
 74,
 80,
 76,
 52,
 56,
 74,
 74,
 74,

In [607]:
%%time
[sum(labels==label) for label in np.unique(labels)]

Wall time: 10min


[12506,
 4692,
 3994,
 3363,
 3526,
 2409,
 2297,
 1601,
 1526,
 1378,
 1298,
 1081,
 1006,
 1007,
 714,
 868,
 870,
 645,
 808,
 807,
 612,
 722,
 754,
 747,
 539,
 656,
 580,
 577,
 679,
 564,
 594,
 907,
 449,
 502,
 452,
 104,
 431,
 410,
 388,
 400,
 369,
 367,
 384,
 347,
 345,
 342,
 314,
 273,
 314,
 316,
 355,
 281,
 336,
 293,
 271,
 246,
 263,
 258,
 254,
 243,
 229,
 291,
 182,
 155,
 137,
 193,
 186,
 197,
 196,
 189,
 191,
 187,
 203,
 178,
 192,
 169,
 173,
 153,
 157,
 199,
 220,
 164,
 162,
 155,
 126,
 152,
 56,
 100,
 141,
 142,
 190,
 117,
 138,
 133,
 135,
 131,
 134,
 122,
 130,
 109,
 132,
 135,
 135,
 120,
 119,
 116,
 110,
 91,
 114,
 114,
 79,
 98,
 112,
 74,
 110,
 67,
 126,
 106,
 110,
 74,
 103,
 102,
 101,
 100,
 85,
 99,
 72,
 99,
 118,
 95,
 98,
 84,
 27,
 99,
 95,
 97,
 97,
 92,
 91,
 103,
 71,
 95,
 87,
 99,
 95,
 33,
 85,
 65,
 121,
 78,
 83,
 81,
 79,
 101,
 79,
 61,
 79,
 78,
 42,
 84,
 78,
 68,
 57,
 63,
 76,
 74,
 80,
 76,
 52,
 56,
 74,
 74,
 74,

In [680]:
%%time
pd.DataFrame({'label_counts':labels})['label_counts'].value_counts().sort_index().values

Wall time: 5 ms


array([12506,  4692,  3994, ...,     1,     1,     1], dtype=int64)

In [624]:
index = pd.DataFrame({'label_counts':labels})['label_counts'].value_counts().index

In [627]:
clusters1 = pd.DataFrame({pd.DataFrame({'label_counts':labels})['label_counts'].value_counts(),
                         'latitude':cluster_centers[ind, 0] for ind in index,
                         'longitude':cluster_centers[ind, 1] for ind in index})

SyntaxError: invalid syntax (<ipython-input-627-fa79aef164cd>, line 2)

In [667]:
clusters1 = pd.DataFrame(pd.DataFrame({'label_counts':labels})['label_counts'].value_counts(), \
                         pd.Series({cluster_centers[ind, 0] for ind in index}),
                         pd.Series({cluster_centers[ind, 1] for ind in index}))

TypeError: 'set' type is unordered

In [675]:
pd.Series({'long':[cluster_centers[ind, 0] for ind in index]})

long    [40.7177163973, 33.4494380502, 33.4463802704, ...
dtype: object

In [664]:
clusters1

,-92.2433487
36.003193,NaN


In [ ]:
clusters1 = pd.DataFrame({'label_counts':[sum(labels==label) for label in np.unique(labels)],
                         'latitude':cluster_centers[:, 0],
                         'longitude':cluster_centers[:, 1]})

In [646]:
clusters1 = pd.DataFrame({'label_counts':pd.DataFrame({'label_counts':labels})['label_counts'].value_counts().sort_index(),
                         'latitude':cluster_centers[:, 0],
                         'longitude':cluster_centers[:, 1]})

In [647]:
clusters1 

,label_counts,latitude,longitude
0,12506,40.717716,-73.991835
1,4692,33.449438,-112.002140
2,3994,33.446380,-111.901888
3,3363,41.878244,-87.629843
4,3526,37.688682,-122.409330
5,2409,38.886165,-77.048783
6,2297,33.357345,-111.822654
7,1601,33.766636,-84.393289
8,1526,42.363219,-71.073688
9,1378,47.606245,-122.332044
